In [3]:
# Imports 
# Standard DS 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Image analysis
import cv2

### There are 4 Steps to building our CBIR System
1. Define our image descriptor --> are we interested in color, shape, texture, etc?
2. Index our images --> apply chosen descriptor to each image, extract features, write these features to storage.
3. Define our similarity metric --> popular choices are Euclidean, Cosine, chi-squared, etc. We have had some success with Minkowski
4. Searching / output --> we will eventually be building a way for a user to submit a query and get results. FOR NOW, we will create a pairwise matrix of distances for our researchers to validate during testing.

In [23]:
class ColorDescriptors:
    def __init__(self, bins):
        """
        Set the number of bins needed for the 3D Hue-Saturation-Value histogram
        
        ----------
        Paramaters
        ----------
        bins   : tuple of ints representing the number of Hue, Saturation, and Value bins
        
        return : doesn't not return anything
        """
        
        # set the number of histogram bins
        self.bins = bins
        
        
    def describe(self, image):
        """
        Convert provided image into feature vector.
        
        1. convert image to HSV color space
        2. initalize the features vector
        3. determin image size and center
        4. create image masks so image can be scanned in sections
        5. loop over segments to build histogram
        6. return histogram / features vector as image vector
        
        ----------
        Paramaters
        ----------
        image  : ndarry representing an image.
        
        return : list of floating point numbers, an n-dimensional vector 
                 where n = number of bins * number of segments. 
        """
        
        # convert the image to the HSV color space and initialize 
        # the features used to quantify the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        features = []
        
        # get the dimensionsand center of the image
        (h, w) = image.shape[:2]
        (cX, cY) = (int(w*0.5), int(h*0.5))
        
        # split image into segmments, this will allow us to simulate
        # locality in a color distribution
        # Divide into 5 segments, top-left, top-right, bottom-left, 
        # bottom-right, and center
        segments = [ (0, cX, 0, cY), (cX, w, 0, cY), 
                     (cX, w, cY, h), (0, cX, cY, h) ]
        
        # create an elliptical mask for the center segment
        (axesX, axesY) = (int(w*0.75) // 2, int(h*0.75) //2)
        ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)
        
        # loop over the segments
        for (startX, endX, startY, endY) in segments:
            # create the mask for each corner subtracting the elliptical center
            cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
            cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
            cornerMask = cv2.subtract(cornerMask, ellipMask)
            
            # extract a color histogram from the image under the mask and update features
            hist = self.histogram(image, cornerMask)
            features.extend(hist)
            
        # extract a color histogram from the elliptical region and update features
        hist = self.histogram(image, ellipMask)
        features.extend(hist)
        
        # return the feature vector
        return features
        
        
    def histogram(self, image, mask):
        """
        Extract a 3D histogram from the masked region of an image.
        
        ----------
        Parameters
        ----------
        image  : ndarray representing an image
        mask   : ndarray representing a masked region
        
        return : list of floating point numbers, n-dimensional vector representing
                 the values of the HSV histogram for the masked image segment
        """
        
        # extract a 3D color histogram from the masked region of the image,
        # use the supplied number of bins
        hist = cv2.calcHist( [image], [0, 1, 2], mask, self.bins,
                             [0, 180, 0, 256, 0, 256] )
        
        # normalize the histogram
        hist = cv2.normalize(hist, hist).flatten()
        
        # return the histogram
        return hist
    
        

In [24]:
# set bins for Hue, Saturation, and Value
bins = (8, 12, 3)

# initalize the color descriptor
cd = ColorDescriptors(bins)

In [25]:
# set path to image we want to open
img_f_name = 'Paintings/1947.001.png'

# read the image
img = cv2.imread(img_f_name)

# get the features for the image
features = cd.describe(img)

In [29]:
"""
There are 8 hue bins, 12 saturation bins, and 3 value bins so the histogram for each segment should be (8 * 12 * 3) = 288 bins

There are 5 segments so the total feature vector length should be 288 * 5 = 1440
"""

assert len(features) == 8*12*3*5